<a href="https://colab.research.google.com/github/teddy-teem/deep-learning/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Load Datasets
2. Embeded the sentence, into vector <br/>
  What is the capital of France?
3. Build RNN
4. Train
5. Predict


https://www.youtube.com/watch?v=xjzWrPQ66VQ&list=PLKnIA16_Rmvboy8bmDCjwNHgTaYH2puK7&index=13&ab_channel=CampusX

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
torch.manual_seed(42)

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
file_path = '/content/drive/My Drive/Datasets/100_Unique_QA_Dataset.csv'

In [6]:
df = pd.read_csv(file_path)
df.shape

(90, 2)

In [7]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [8]:
# Tokenize
def tokenize(text):
  text = text.lower()
  text= text.replace("?", "")
  text = text.replace("'", "")
  return text.split()


In [9]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

In [10]:
# Vocabulary
vocab = {'<UNK>': 0}


In [11]:
def buildVocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer=tokenize(row['answer'])
  merged_token = tokenized_question + tokenized_answer
  for token in merged_token:
    if token not in vocab:
      vocab[token] = len(vocab)



In [12]:
df.apply(buildVocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [13]:
len(vocab)

324

In [14]:
# convert words to numerical indices
def text_to_indices(text, vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text


In [15]:
text_to_indices("What is the capital of France?", vocab)

[1, 2, 3, 4, 5, 6]

In [59]:
class QADataset(Dataset):
  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, idx):
    numerical_questions = text_to_indices(self.df.iloc[idx]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[idx]['answer'], self.vocab)
    return torch.tensor(numerical_questions, dtype=torch.long), torch.tensor(numerical_answer, dtype=torch.long)

In [60]:
dataset = QADataset(df, vocab)


In [61]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [62]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [63]:
for questions, answers in dataloader:
  print(questions)
  print(answers)


tensor([[  1,   2,   3,  37,  38,  39, 161]])
tensor([[162]])
tensor([[  1,   2,   3, 180, 181, 182, 183]])
tensor([[184]])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]])
tensor([[268]])
tensor([[  1,   2,   3,   4,   5, 279]])
tensor([[280]])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]])
tensor([[6]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]])
tensor([[106]])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]])
tensor([[273]])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]])
tensor([[220]])
tensor([[ 1,  2,  3, 92, 93, 94]])
tensor([[95]])
tensor([[ 42, 200,   2,  14, 201, 202, 203, 204]])
tensor([[205]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 277, 278]])
tensor([[121]])
tensor([[ 10,  11, 189, 158, 190]])
tensor([[191]])
tensor([[ 42, 137,   2, 138,  39, 175, 269]])
tensor([[99]])
tensor([[10,  2,  3, 66,  5, 67]])
tensor([[68]])
tensor([[ 42, 250, 251, 118, 252, 253]])
tensor([[254]])
tensor([[ 10,   2,  62,  63,   3, 283,   5, 284]])
tensor([[28

In [69]:
class SimpleRNN(nn.Module):
  def __init__(self, vocab_size):
    super().__init__();
    self.embading = nn.Embedding(vocab_size, embedding_dim=50);
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embeded_question =  self.embading(question)
    hidden, final  =self.rnn(embeded_question)
    output = self.fc(final.squeeze(0))
    return output;

In [70]:
learing_rate = 0.001
epochs = 20

In [71]:
model = SimpleRNN(len(vocab))


In [72]:
criterion = nn.CrossEntropyLoss()
optimizer =  torch.optim.Adam(model.parameters(), lr=learing_rate)

In [74]:
for epoch in range(epochs):
  total_loss = 0

  for questions, answer in dataloader:

    optimizer.zero_grad()

    output = model(questions)
    output.shape
    #loss -> output shape (1, 324) - (1)
    loss = criterion(output, answer[0])

    loss.backward()

    optimizer.step()

    total_loss += loss.item() #Total loss, so it's not avrg lost
  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")



Epoch: 1, Loss: 10.188361
Epoch: 2, Loss: 8.962585
Epoch: 3, Loss: 7.872274
Epoch: 4, Loss: 7.031404
Epoch: 5, Loss: 6.333329
Epoch: 6, Loss: 5.707768
Epoch: 7, Loss: 5.167884
Epoch: 8, Loss: 4.677187
Epoch: 9, Loss: 4.284064
Epoch: 10, Loss: 3.918984
Epoch: 11, Loss: 3.601120
Epoch: 12, Loss: 3.323801
Epoch: 13, Loss: 3.062444
Epoch: 14, Loss: 2.840604
Epoch: 15, Loss: 2.628682
Epoch: 16, Loss: 2.441478
Epoch: 17, Loss: 2.267083
Epoch: 18, Loss: 2.114633
Epoch: 19, Loss: 1.974969
Epoch: 20, Loss: 1.843546


In [75]:
def predict(model, question, thresold=0.5):
  # convert questions to numbers
  numerical_question =  text_to_indices(question, vocab)
  question_tensor = torch.tensor(numerical_question, dtype=torch.long).unsqueeze(0)
  ouptut = model(question_tensor)
  probs = torch.nn.functional.softmax(ouptut, dim=1)
  value, index = torch.max(probs, dim=1)

  if value < thresold:
    print("I don't know the answer")

  print(value, index)
  # predict

In [84]:
predict(model, "What is Capital of Bangladesh?")

tensor([0.1675], grad_fn=<MaxBackward0>) tensor([287])


In [85]:
list(vocab.keys())[287]

'mexicocity'